[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danpele/Time-Series-Analysis/blob/main/chapter8_seminar_notebook.ipynb)

---

# Chapter 8 Seminar: Modern Extensions - Practice

**Course:** Time Series Analysis and Forecasting  
**Program:** Bachelor program, Faculty of Cybernetics, Statistics and Economic Informatics, Bucharest University of Economic Studies, Romania  
**Academic Year:** 2025-2026

---

## Seminar Objectives

In this practical seminar, you will:
1. Estimate the Hurst exponent and identify long memory
2. Apply feature engineering for time series ML
3. Build and evaluate Random Forest models
4. Implement simple LSTM networks
5. Compare classical vs ML approaches

## Setup

In [ ]:
# Install required packages (for Colab)
import sys
if 'google.colab' in sys.modules:
    !pip install nolds arch yfinance -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

try:
    import nolds
    HAS_NOLDS = True
except ImportError:
    HAS_NOLDS = False
    print("nolds not installed - using manual Hurst calculation")

try:
    import yfinance as yf
    HAS_YF = True
except ImportError:
    HAS_YF = False

plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['axes.grid'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['legend.frameon'] = False

COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666'}

print("Setup complete!")

## Exercise 1: Hurst Exponent Estimation

**Task:** Estimate the Hurst exponent for different types of series and interpret the results.

In [ ]:
def hurst_rs(series, min_window=10):
    """Calculate Hurst exponent using R/S analysis."""
    n = len(series)
    max_k = int(np.floor(n / min_window))
    
    rs_values = []
    n_values = []
    
    for k in range(2, max_k + 1):
        size = n // k
        rs_k = []
        
        for i in range(k):
            subseries = series[i * size:(i + 1) * size]
            mean_sub = np.mean(subseries)
            cumdev = np.cumsum(subseries - mean_sub)
            R = np.max(cumdev) - np.min(cumdev)
            S = np.std(subseries, ddof=1)
            if S > 0:
                rs_k.append(R / S)
        
        if rs_k:
            rs_values.append(np.mean(rs_k))
            n_values.append(size)
    
    log_n = np.log(n_values)
    log_rs = np.log(rs_values)
    
    slope, _ = np.polyfit(log_n, log_rs, 1)
    return slope

# Generate different types of series
np.random.seed(42)
n = 1000

# 1. Random walk (H ≈ 0.5)
random_walk = np.cumsum(np.random.randn(n))

# 2. Mean-reverting (H < 0.5)
mean_rev = np.zeros(n)
for i in range(1, n):
    mean_rev[i] = -0.5 * mean_rev[i-1] + np.random.randn()

# 3. Trending/Persistent (H > 0.5)
persistent = np.zeros(n)
for i in range(1, n):
    persistent[i] = 0.7 * persistent[i-1] + np.sign(persistent[i-1]) * 0.3 + np.random.randn() * 0.5
persistent = np.cumsum(persistent)

print("Hurst Exponent Estimation")
print("=" * 50)

series_dict = {
    'Random Walk': random_walk,
    'Mean-Reverting': mean_rev,
    'Persistent': persistent
}

for name, series in series_dict.items():
    h = hurst_rs(series)
    if HAS_NOLDS:
        h_nolds = nolds.hurst_rs(series)
        print(f"{name:20s}: H = {h:.3f} (manual), {h_nolds:.3f} (nolds)")
    else:
        print(f"{name:20s}: H = {h:.3f}")

In [ ]:
# Visualize the three types of series
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

h_vals = {}
for ax, (name, series) in zip(axes, series_dict.items()):
    h = hurst_rs(series)
    h_vals[name] = h
    
    if h < 0.5:
        color = COLORS['green']
        behavior = 'Mean-reverting'
    elif h > 0.5:
        color = COLORS['red']
        behavior = 'Trending'
    else:
        color = COLORS['gray']
        behavior = 'Random Walk'
    
    ax.plot(series, color=color, linewidth=0.8)
    ax.set_title(f"{name}\nH = {h:.3f} ({behavior})", fontweight='bold')
    ax.set_xlabel('Time')

plt.tight_layout()
plt.show()

print("\nInterpretation Guide:")
print("  H < 0.5: Anti-persistent (mean-reverting) - good for mean-reversion strategies")
print("  H = 0.5: Random walk (no memory) - unpredictable")
print("  H > 0.5: Persistent (trending) - good for momentum strategies")

## Exercise 2: Long Memory in Real Financial Data

**Task:** Download Bitcoin data and test for long memory in returns vs volatility.

In [ ]:
# Get Bitcoin data
if HAS_YF:
    btc = yf.download('BTC-USD', start='2020-01-01', end='2024-12-31', progress=False)
    # Handle MultiIndex columns from newer yfinance versions
    if isinstance(btc.columns, pd.MultiIndex):
        btc.columns = btc.columns.droplevel(1)
    # Ensure we get a Series and extract values
    close_prices = btc['Close'].squeeze()
    price = close_prices.dropna().values
    returns = np.diff(np.log(price)) * 100  # Log returns in %
else:
    # Simulate if no yfinance
    np.random.seed(123)
    n = 1000
    returns = np.random.randn(n) * 3  # Simulate returns
    # Add volatility clustering
    vol = np.ones(n)
    for i in range(1, n):
        vol[i] = 0.1 + 0.85 * vol[i-1] + 0.1 * returns[i-1]**2
    returns = returns * np.sqrt(vol)
    price = 10000 * np.exp(np.cumsum(returns / 100))

print(f"Data loaded: {len(returns)} observations")

# Calculate volatility proxies
abs_returns = np.abs(returns)
squared_returns = returns ** 2

print("\nLong Memory Analysis: Returns vs Volatility")
print("=" * 60)

In [ ]:
# Compare Hurst exponents
h_returns = hurst_rs(returns)
h_abs = hurst_rs(abs_returns)
h_sq = hurst_rs(squared_returns)

print(f"{'Series':<25} {'Hurst':>10} {'Memory Type':>20}")
print("-" * 60)

for name, h in [('Returns', h_returns), ('|Returns|', h_abs), ('Returns²', h_sq)]:
    if h < 0.45:
        mem_type = 'Anti-persistent'
    elif h > 0.55:
        mem_type = 'Long memory (persistent)'
    else:
        mem_type = 'Short memory (~random)'
    print(f"{name:<25} {h:>10.3f} {mem_type:>20}")

print("\n" + "="*60)
print("KEY FINDING: Volatility has long memory, returns do not!")
print("This is a stylized fact used in FIGARCH models.")

In [ ]:
# Visualize ACF decay comparison
from statsmodels.tsa.stattools import acf

max_lag = 100
acf_returns = acf(returns, nlags=max_lag, fft=True)
acf_abs = acf(abs_returns, nlags=max_lag, fft=True)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Returns ACF (fast decay)
axes[0].bar(range(max_lag+1), acf_returns, color=COLORS['blue'], alpha=0.7, width=0.8)
axes[0].axhline(y=1.96/np.sqrt(len(returns)), color='red', linestyle='--', alpha=0.5)
axes[0].axhline(y=-1.96/np.sqrt(len(returns)), color='red', linestyle='--', alpha=0.5)
axes[0].set_title(f'ACF of Returns (H = {h_returns:.3f})\nFast Decay = Short Memory', fontweight='bold')
axes[0].set_xlabel('Lag')
axes[0].set_xlim(-1, max_lag)

# Absolute returns ACF (slow decay)
axes[1].bar(range(max_lag+1), acf_abs, color=COLORS['red'], alpha=0.7, width=0.8)
axes[1].axhline(y=1.96/np.sqrt(len(abs_returns)), color='red', linestyle='--', alpha=0.5)
axes[1].axhline(y=-1.96/np.sqrt(len(abs_returns)), color='red', linestyle='--', alpha=0.5)
axes[1].set_title(f'ACF of |Returns| (H = {h_abs:.3f})\nSlow Decay = Long Memory', fontweight='bold')
axes[1].set_xlabel('Lag')
axes[1].set_xlim(-1, max_lag)

plt.tight_layout()
plt.show()

## Exercise 3: Feature Engineering for Time Series ML

**Task:** Create lag features and rolling statistics for ML models.

In [ ]:
def create_features(series, lags=7, rolling_windows=[7, 14, 30]):
    """
    Create lag and rolling features for time series ML.
    
    Parameters:
    - series: 1D array of values
    - lags: number of lag features
    - rolling_windows: list of window sizes for rolling stats
    
    Returns:
    - DataFrame with features
    """
    df = pd.DataFrame({'y': series})
    
    # Lag features
    for i in range(1, lags + 1):
        df[f'lag_{i}'] = df['y'].shift(i)
    
    # Rolling statistics
    for w in rolling_windows:
        df[f'rolling_mean_{w}'] = df['y'].shift(1).rolling(window=w).mean()
        df[f'rolling_std_{w}'] = df['y'].shift(1).rolling(window=w).std()
        df[f'rolling_min_{w}'] = df['y'].shift(1).rolling(window=w).min()
        df[f'rolling_max_{w}'] = df['y'].shift(1).rolling(window=w).max()
    
    # Target: next value
    df['target'] = df['y']
    
    # Drop rows with NaN
    df = df.dropna()
    
    return df

# Create features from price data
features_df = create_features(price, lags=7, rolling_windows=[7, 14, 30])
print(f"Feature DataFrame shape: {features_df.shape}")
print(f"\nFeatures created:")
print(features_df.columns.tolist())

In [ ]:
# Display sample of features
print("\nSample of engineered features:")
display_cols = ['y', 'lag_1', 'lag_2', 'rolling_mean_7', 'rolling_std_7', 'target']
print(features_df[display_cols].head(10).round(2))

## Exercise 4: Random Forest with Time Series Cross-Validation

**Task:** Train Random Forest and evaluate using TimeSeriesSplit.

In [ ]:
# Prepare data
feature_cols = [c for c in features_df.columns if c not in ['y', 'target']]
X = features_df[feature_cols].values
y = features_df['target'].values

print(f"Features: {len(feature_cols)}")
print(f"Samples: {len(X)}")

# Time Series Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)

# Store results
cv_results = []

print("\nTime Series Cross-Validation")
print("=" * 60)

for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Train model
    rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    # Predict
    y_pred = rf.predict(X_test)
    
    # Metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    
    # Direction accuracy
    y_test_diff = np.diff(y_test)
    y_pred_diff = np.diff(y_pred)
    dir_acc = np.mean(np.sign(y_test_diff) == np.sign(y_pred_diff)) * 100
    
    cv_results.append({'fold': fold+1, 'rmse': rmse, 'mae': mae, 'dir_acc': dir_acc})
    print(f"Fold {fold+1}: Train={len(train_idx):5d}, Test={len(test_idx):4d} | RMSE={rmse:10.2f}, Dir.Acc={dir_acc:.1f}%")

# Summary
results_df = pd.DataFrame(cv_results)
print("\n" + "-"*60)
print(f"Average RMSE: {results_df['rmse'].mean():.2f} ± {results_df['rmse'].std():.2f}")
print(f"Average Direction Accuracy: {results_df['dir_acc'].mean():.1f}%")

In [ ]:
# Feature Importance Analysis
rf_final = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_final.fit(X, y)

importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_final.feature_importances_
}).sort_values('importance', ascending=True)

# Plot top 15 features
fig, ax = plt.subplots(figsize=(10, 6))
top_features = importance_df.tail(15)
colors = [COLORS['blue'] if 'lag' in f else COLORS['orange'] for f in top_features['feature']]
ax.barh(top_features['feature'], top_features['importance'], color=colors)
ax.set_xlabel('Importance')
ax.set_title('Top 15 Feature Importances\n(Blue=Lag, Orange=Rolling)', fontweight='bold')
plt.tight_layout()
plt.show()

print("\nInsight: Lag features (especially lag_1) are most important.")
print("Rolling statistics add value for capturing trends.")

## Exercise 5: Simple LSTM Implementation

**Task:** Build a basic LSTM model for price prediction.

In [ ]:
# Try to import TensorFlow
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from tensorflow.keras.callbacks import EarlyStopping
    HAS_TF = True
    print(f"TensorFlow version: {tf.__version__}")
except ImportError:
    HAS_TF = False
    print("TensorFlow not installed. LSTM exercises will be skipped.")

In [ ]:
if HAS_TF:
    # Prepare data for LSTM
    def create_sequences(data, seq_length):
        """Create sequences for LSTM input."""
        X, y = [], []
        for i in range(seq_length, len(data)):
            X.append(data[i-seq_length:i, 0])
            y.append(data[i, 0])
        return np.array(X), np.array(y)
    
    # Scale data
    scaler = MinMaxScaler(feature_range=(0, 1))
    price_scaled = scaler.fit_transform(price.reshape(-1, 1))
    
    # Create sequences
    SEQ_LENGTH = 30
    X_lstm, y_lstm = create_sequences(price_scaled, SEQ_LENGTH)
    
    # Reshape for LSTM [samples, timesteps, features]
    X_lstm = X_lstm.reshape((X_lstm.shape[0], X_lstm.shape[1], 1))
    
    # Train/test split (time-based)
    train_size = int(len(X_lstm) * 0.8)
    X_train_lstm = X_lstm[:train_size]
    y_train_lstm = y_lstm[:train_size]
    X_test_lstm = X_lstm[train_size:]
    y_test_lstm = y_lstm[train_size:]
    
    print(f"LSTM Input shape: {X_lstm.shape}")
    print(f"Training samples: {len(X_train_lstm)}")
    print(f"Test samples: {len(X_test_lstm)}")
else:
    print("Skipping LSTM - TensorFlow not available")

In [ ]:
if HAS_TF:
    # Build LSTM model
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, 1)),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='mse')
    model.summary()
    
    # Early stopping
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Train
    print("\nTraining LSTM...")
    history = model.fit(
        X_train_lstm, y_train_lstm,
        epochs=50,
        batch_size=32,
        validation_split=0.1,
        callbacks=[early_stop],
        verbose=1
    )

In [ ]:
if HAS_TF:
    # Evaluate LSTM
    y_pred_lstm = model.predict(X_test_lstm, verbose=0)
    
    # Inverse transform
    y_test_actual = scaler.inverse_transform(y_test_lstm.reshape(-1, 1)).flatten()
    y_pred_actual = scaler.inverse_transform(y_pred_lstm).flatten()
    
    # Metrics
    rmse_lstm = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))
    mae_lstm = mean_absolute_error(y_test_actual, y_pred_actual)
    
    # Direction accuracy
    y_test_diff = np.diff(y_test_actual)
    y_pred_diff = np.diff(y_pred_actual)
    dir_acc_lstm = np.mean(np.sign(y_test_diff) == np.sign(y_pred_diff)) * 100
    
    print("LSTM Results")
    print("=" * 40)
    print(f"RMSE: {rmse_lstm:.2f}")
    print(f"MAE: {mae_lstm:.2f}")
    print(f"Direction Accuracy: {dir_acc_lstm:.1f}%")

In [ ]:
if HAS_TF:
    # Plot predictions
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    # Full test period
    axes[0].plot(y_test_actual, color=COLORS['blue'], label='Actual', linewidth=1)
    axes[0].plot(y_pred_actual, color=COLORS['red'], label='LSTM Predicted', linewidth=1, alpha=0.8)
    axes[0].set_title('LSTM Predictions vs Actual (Test Set)', fontweight='bold')
    axes[0].legend()
    axes[0].set_ylabel('Price ($)')
    
    # Last 100 days detail
    axes[1].plot(y_test_actual[-100:], color=COLORS['blue'], label='Actual', linewidth=1.5)
    axes[1].plot(y_pred_actual[-100:], color=COLORS['red'], label='LSTM Predicted', linewidth=1.5, alpha=0.8)
    axes[1].set_title('Last 100 Days (Detail)', fontweight='bold')
    axes[1].legend()
    axes[1].set_xlabel('Days')
    axes[1].set_ylabel('Price ($)')
    
    plt.tight_layout()
    plt.show()

## Exercise 6: Model Comparison

**Task:** Compare Random Forest vs LSTM.

In [ ]:
print("Model Comparison: Random Forest vs LSTM")
print("=" * 60)
print(f"{'Metric':<25} {'Random Forest':>15} {'LSTM':>15}")
print("-" * 60)

# Use last fold for RF comparison
rf_rmse = results_df['rmse'].iloc[-1]
rf_dir = results_df['dir_acc'].iloc[-1]

print(f"{'RMSE':<25} {rf_rmse:>15.2f}", end='')
if HAS_TF:
    print(f" {rmse_lstm:>15.2f}")
else:
    print(f" {'N/A':>15}")

print(f"{'Direction Accuracy (%)':<25} {rf_dir:>15.1f}", end='')
if HAS_TF:
    print(f" {dir_acc_lstm:>15.1f}")
else:
    print(f" {'N/A':>15}")

print("\n" + "=" * 60)
print("\nKey Insights:")
print("• Random Forest: Fast, interpretable, good with engineered features")
print("• LSTM: Better for complex patterns, needs more data and tuning")
print("• Direction accuracy ~50% suggests markets are nearly efficient")

## Practice Problems

### Problem 1: Hurst Exponent Interpretation

You estimate Hurst exponent for three series:
- Series A: H = 0.72
- Series B: H = 0.48  
- Series C: H = 0.31

**Question:** For which series would momentum strategies work best?

In [ ]:
print("Problem 1 Solution")
print("=" * 50)

series = {'A': 0.72, 'B': 0.48, 'C': 0.31}

for name, h in series.items():
    if h > 0.5:
        behavior = "Persistent (trending)"
        strategy = "Momentum ✓"
    elif h < 0.5:
        behavior = "Anti-persistent (mean-reverting)"
        strategy = "Mean-reversion"
    else:
        behavior = "Random walk"
        strategy = "None (unpredictable)"
    
    print(f"Series {name}: H = {h} → {behavior}")
    print(f"           Best strategy: {strategy}\n")

print("Answer: Series A (H = 0.72) - strongest persistence")

### Problem 2: Feature Selection

You have these features:
- lag_1, lag_2, lag_3
- rolling_mean_7, rolling_std_7
- future_return (tomorrow's return)

**Question:** Which feature should NOT be included in training?

In [ ]:
print("Problem 2 Solution")
print("=" * 50)

print("Answer: future_return should NOT be included!")
print("\nReason: DATA LEAKAGE")
print("• future_return contains information from the future")
print("• Including it would artificially inflate model performance")
print("• In real trading, we don't know tomorrow's return today")
print("\nValid features use only past information:")
print("• lag_1, lag_2, lag_3 ✓ (past values)")
print("• rolling_mean_7, rolling_std_7 ✓ (past statistics)")

### Problem 3: LSTM Architecture

Your LSTM has:
- Input: sequences of 60 days
- Output: predicting price 5 days ahead

**Question:** What shape should the output layer have?

In [ ]:
print("Problem 3 Solution")
print("=" * 50)

print("For multi-step prediction (5 days ahead):")
print("\nOption 1: Direct Multi-Output")
print("  Dense(5) - one output per day")
print("  Predicts: [day+1, day+2, day+3, day+4, day+5]")
print("\nOption 2: Recursive")
print("  Dense(1) - predict day+1")
print("  Feed prediction back as input, repeat 5 times")
print("\nOption 3: Sequence-to-Sequence")
print("  LSTM encoder → LSTM decoder")
print("  Output sequence of length 5")

print("\nRecommendation: Direct Multi-Output (Dense(5)) is simplest")

## Summary

### Key Takeaways

1. **Long Memory Detection**
   - Hurst exponent: H > 0.5 = trending, H < 0.5 = mean-reverting
   - Returns: ~0.5 (no memory), Volatility: ~0.7-0.8 (long memory)

2. **Feature Engineering**
   - Lag features capture autocorrelation
   - Rolling statistics capture trends
   - Always use shifted values to avoid leakage

3. **Time Series Cross-Validation**
   - Never use standard k-fold
   - TimeSeriesSplit maintains temporal order
   - Walk-forward validation for realistic evaluation

4. **Model Selection**
   - Random Forest: Fast, interpretable, good baseline
   - LSTM: Complex patterns, more data needed
   - Direction accuracy near 50% suggests market efficiency

### Practical Workflow
1. Test for long memory (Hurst exponent)
2. Engineer appropriate features
3. Use time series cross-validation
4. Compare multiple models
5. Evaluate with appropriate metrics (RMSE, direction accuracy)